# Обработка файлов изображений

Для анализа изображений использована старая версия Adobe Bridge. Она позволяет не только просматривать изображения, но и отбирать их по сложным критериям, например соотношению размеров сторон.  Аналогичное исследование можно было бы сделать и средствами PIL. 

По видимому изображения взяты с сайта avito. В очень многих случаях есть логотип этого сервиса объявлений. 

Исходные изображения сжаты до ограничивающего размера приблизительно 640х480. Исследование исходных файлов показало большое разнообразие аспекта изображений (соотношения длинной и короткой сторон). Подавляющее большинство - картинки с горизонтальной ориентацией и соотношением сторон 3:4  Однако встречаются горизонтальные изображения с аспектом 21:9. Их небольшое количество - примерно 4-5 на класс, я решил отбросить их. 

Однако - 10..15% изображений в каждом классе - картинки с вертикальной ориентацией изображения. И там встречаются изображения с аспектом и 21:9. При подготовке набора изображений - они были сжаты до размера 271:480. 

Особенность алгоритмов CV состоит в том, что при изменении масштаба происходит растягивание изображения по короткой оси. Если буду подгонять все под горизонтальный формат 3:4  - такая картинка растянется по горизонтали довольно существенно.  Хочу этого избежать В дальнейшем думаю провести сравнение того, какой формат изображения лучше для НС - горизонтальный или квадрат. 

Попытка визуального редактирования средствами PixelmatorPro показала, что физики, фотографируя автомобиль выбирают формат 21:9 по недомыслию и размешают машинку в центре кадра. Сверху и снизу остается пустое пространство. Думаю, что можно будет просто обрезать вертикальные картинки до аспекта центрированный квадрат.  Этот рабочий файл как раз и обрабатывает картинки. 



In [9]:
import os, path, PIL
from PIL import Image

In [10]:
'''
work_dir='./Images/train/1/Vert/'
save_dir=work_dir+'done/'

DirItems = os.scandir(work_dir)
cnt = 0
for item in DirItems:
    if item.is_file :
        try:
            with Image.open(work_dir+item.name) as im:
                orig_w=im.size[0]
                orig_h=im.size[1]
                hh = (orig_h-orig_w)//2
                cnt +=1
                # print(item.name, orig_w, orig_h, hh)
                cropped = im.crop((0,hh,orig_w,orig_h-hh))
                cropped.save(save_dir+item.name)    
        except OSError:
            print("cannot convert", work_dir+item.name)
print(cnt,'files cropped')
'''

        


'\nwork_dir=\'./Images/train/1/Vert/\'\nsave_dir=work_dir+\'done/\'\n\nDirItems = os.scandir(work_dir)\ncnt = 0\nfor item in DirItems:\n    if item.is_file :\n        try:\n            with Image.open(work_dir+item.name) as im:\n                orig_w=im.size[0]\n                orig_h=im.size[1]\n                hh = (orig_h-orig_w)//2\n                cnt +=1\n                # print(item.name, orig_w, orig_h, hh)\n                cropped = im.crop((0,hh,orig_w,orig_h-hh))\n                cropped.save(save_dir+item.name)    \n        except OSError:\n            print("cannot convert", work_dir+item.name)\nprint(cnt,\'files cropped\')\n'

Обработка первого каталога показала, что 194 из 1610 были вертикальными. Из 194 последующей правки вручную потребовали 22 изображения, на которых машинка была расположена выше центра. 

После нескольких экспериментов было принято решение не делать набор картинок большого размера, предусматривающий последующее уменьшение в каждой эпохе обучения. Реализована идея уменьшения картинок до размера, соответствующего входному слою сетей (квадрат 224 пикселя). Учитывая, что картинки имеют разный размер по ширине и высоте было принято решение дополнить их черным цветом до квадрата.  Большие в высоту картинки обрезались снизу в зависимости от размера.  Тестовые картинки были сконвертированы по тому же алгоритму. 

Просмотр картинок показал, что в папке 0 содержались несколько фотографий старенького а/м марки Москвич кирпичного цвета. Они были удалены.  В папке про Ford Focus были замечены фотографии битых машин после аварии. Они пока не удалялись.  

In [11]:
from PIL.ImageOps import pad
source = [ './Images/train_b/0/','./Images/train_b/1/', './Images/train_b/2/', './Images/train_b/3/', './Images/train_b/4/', 
            './Images/train_b/5/', './Images/train_b/6/', './Images/train_b/7/', './Images/train_b/8/', './Images/train_b/9/', './Images/test_upload_b/']
dest = ['./Images/train/0/', './Images/train/1/', './Images/train/2/', './Images/train/3/', './Images/train/4/', './Images/train/5/',
            './Images/train/6/', './Images/train/7/', './Images/train/8/', './Images/train/9/', './Images/test/' ]

In [12]:
'''
def convert_image(src, dst, nm, size):
    # print(src+nm, dst+nm,  size )
    try:
        with Image.open(src+nm) as im:
            # print(src+nm)
            orig_w=im.size[0]
            orig_h=im.size[1]
            if orig_h > orig_w: 
                aspect_ratio = orig_h/orig_w
                hh = 0 if aspect_ratio < 1.4 else (int(orig_h/9) if aspect_ratio < 1.6 else (orig_h-orig_w)//2)
                cropped = im.crop((0,0,orig_w,orig_h-hh))
            else : cropped = im
            # cropped.show()
            padded = pad(cropped,(size,size),centering=(0.5,0.5))
            # padded.show()
            padded.save(dst+nm)
            return 1    
    except OSError:
        print("cannot convert", src+nm)
        return 0
'''

'\ndef convert_image(src, dst, nm, size):\n    # print(src+nm, dst+nm,  size )\n    try:\n        with Image.open(src+nm) as im:\n            # print(src+nm)\n            orig_w=im.size[0]\n            orig_h=im.size[1]\n            if orig_h > orig_w: \n                aspect_ratio = orig_h/orig_w\n                hh = 0 if aspect_ratio < 1.4 else (int(orig_h/9) if aspect_ratio < 1.6 else (orig_h-orig_w)//2)\n                cropped = im.crop((0,0,orig_w,orig_h-hh))\n            else : cropped = im\n            # cropped.show()\n            padded = pad(cropped,(size,size),centering=(0.5,0.5))\n            # padded.show()\n            padded.save(dst+nm)\n            return 1    \n    except OSError:\n        print("cannot convert", src+nm)\n        return 0\n'

In [13]:
'''
for i in range(0,len(source)):
    src = source[i]; dst = dest[i]
    DirItems = os.scandir(src)
    cnt = 0
    for item in DirItems:
        if item.is_file :
            cnt += convert_image(src,dst,item.name,224)    
    print(cnt,'files cropped from',src,'to',dst)
'''

"\nfor i in range(0,len(source)):\n    src = source[i]; dst = dest[i]\n    DirItems = os.scandir(src)\n    cnt = 0\n    for item in DirItems:\n        if item.is_file :\n            cnt += convert_image(src,dst,item.name,224)    \n    print(cnt,'files cropped from',src,'to',dst)\n"

Операция преобразования изображений заняла 3 минуты 48 сек на локальной машине при чтении и записи на SSD.  Мне интересно - сколько из этого приходится на обработку при сжатии картинки и какой выигрыш даст во времени чтение этого набора данных без обработки.  
Есть выигрыш в размере архива - 181 МБ вместо исходных 1.7 ГБ. 

In [14]:
def stretch_image(src, dst, nm, size):
    # print(src+nm, dst+nm,  size )
    try:
        with Image.open(src+nm) as im:
            # print(src+nm)
            orig_w=im.size[0]
            orig_h=im.size[1]
            if orig_h > orig_w: 
                aspect_ratio = orig_h/orig_w
                hh = 0 if aspect_ratio < 1.4 else (int(orig_h/9) if aspect_ratio < 1.6 else (orig_h-orig_w)//2)
                cropped = im.crop((0,0,orig_w,orig_h-hh))
            else : cropped = im
            stretched = cropped.resize((size,size))
            stretched.save(dst+nm)
            return 1    
    except OSError:
        print("cannot convert", src+nm)
        return 0

In [15]:
# stretch_image(source[0],dest[0],'1562.jpg',224)
for i in range(0,len(source)):
    src = source[i]; dst = dest[i]
    DirItems = os.scandir(src)
    cnt = 0
    for item in DirItems:
        if item.is_file :
            cnt += stretch_image(src,dst,item.name,380)    
    print(cnt,'files cropped from',src,'to',dst)


cannot convert ./Images/train_b/0/.DS_Store
1609 files cropped from ./Images/train_b/0/ to ./Images/train/0/
1971 files cropped from ./Images/train_b/1/ to ./Images/train/1/
1458 files cropped from ./Images/train_b/2/ to ./Images/train/2/
1528 files cropped from ./Images/train_b/3/ to ./Images/train/3/
1400 files cropped from ./Images/train_b/4/ to ./Images/train/4/
1631 files cropped from ./Images/train_b/5/ to ./Images/train/5/
1733 files cropped from ./Images/train_b/6/ to ./Images/train/6/
1207 files cropped from ./Images/train_b/7/ to ./Images/train/7/
1765 files cropped from ./Images/train_b/8/ to ./Images/train/8/
1255 files cropped from ./Images/train_b/9/ to ./Images/train/9/
6675 files cropped from ./Images/test_upload_b/ to ./Images/test/
